In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# df = pd.read_csv("../dataset/hr_train.csv")
# df.head()
# df.info()
# df.sales.unique()
# df.salary.unique()
# df.groupby(['salary', 'sales']).size()

# pd.get_dummies(df.salary)
# df = df.join(pd.get_dummies(df.salary))
# del df['salary']

# pd.get_dummies(df.sales)  # 读日编码？？？
# df = df.rename(columns={'sales':'department'})
# df = df.join(pd.get_dummies(df.department))
# del df['department']
# df.to_csv("../dataset/hr_train_ok.csv", index=False)

In [3]:
# df = pd.read_csv("../dataset/hr_test_ok.csv")
df = pd.read_csv("../dataset/hr_train_ok.csv")
del df['id']
df.head(3)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,high,low,...,IT,RandD,accounting,hr,management,marketing,product_mng,sales,support,technical
0,0.75,0.81,5,227,5,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0.13,0.91,2,149,5,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0.67,0.74,3,226,3,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11999 entries, 0 to 11998
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     11999 non-null  float64
 1   last_evaluation        11999 non-null  float64
 2   number_project         11999 non-null  int64  
 3   average_montly_hours   11999 non-null  int64  
 4   time_spend_company     11999 non-null  int64  
 5   Work_accident          11999 non-null  int64  
 6   left                   11999 non-null  int64  
 7   promotion_last_5years  11999 non-null  int64  
 8   high                   11999 non-null  int64  
 9   low                    11999 non-null  int64  
 10  medium                 11999 non-null  int64  
 11  IT                     11999 non-null  int64  
 12  RandD                  11999 non-null  int64  
 13  accounting             11999 non-null  int64  
 14  hr                     11999 non-null  int64  
 15  ma

In [5]:
df.shape

(11999, 21)

In [6]:
df.left.unique()

array([1, 0], dtype=int64)

In [7]:
df.left.value_counts()

0    9129
1    2870
Name: left, dtype: int64

In [8]:
df.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'high', 'low', 'medium', 'IT', 'RandD',
       'accounting', 'hr', 'management', 'marketing', 'product_mng', 'sales',
       'support', 'technical'],
      dtype='object')

In [9]:
test = df.left   # 输出的不是pd，只是列
test

0        1
1        0
2        0
3        0
4        1
        ..
11994    0
11995    0
11996    0
11997    0
11998    0
Name: left, Length: 11999, dtype: int64

In [10]:
Y_data = df.left.values.reshape(-1,1)  # 转换成 pd type array
Y_data

array([[1],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]], dtype=int64)

In [11]:
# Y = torch.from_numpy(Y_data).type(torch.float32)  # 转换成tensor
Y = torch.from_numpy(Y_data).type(torch.FloatTensor)
Y.shape
# Y.size()

torch.Size([11999, 1])

In [12]:
X_columns = [c for c in df.columns if c != 'left']
X_data = df[X_columns]  
# X = torch.from_numpy(X_data.values).type(torch.FloatTensor)
X = torch.from_numpy(X_data.values).type(torch.float32)
X.shape

torch.Size([11999, 20])

In [13]:
import torch.nn.functional as F  # 避免relu和sigmoid的初始化，可以直接调用

class Model(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(20, 64)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(64, 64)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
    def forward(self, input):
        x = F.relu( self.liner_1(input) )
        x = F.relu( self.liner_2(x) )
        x = F.sigmoid( self.liner_3(x) )
        return x

# 更加高级语言法，等价于Model
class Model_high(nn.Module):
    def __init__(self):
        super().__init__()      # 继承父类中所有的属性
        self.liner_1 = nn.Linear(20, 64)   # X.size(1)= 20； 定义64个后续中间层
        self.liner_2 = nn.Linear(64, 64)   # 接着，再定义64个后续中间层
        self.liner_3 = nn.Linear(64, 1)   # 接着，再定义64个后续中间层
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    def forward(self, input):
        x = self.Liner_1(input)
        x = self.relu(x)
        x = self.Liner_2(x)
        x = self.relu(x)
        x = self.Liner_3(x)
        x = self.sigmoid(x)
        return x

In [14]:
lr = 0.0001
def get_model():
    model = Model()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    return model, opt

In [15]:
model, optim = get_model()
print( model, "\n" )
print( optim )

Model(
  (liner_1): Linear(in_features=20, out_features=64, bias=True)
  (liner_2): Linear(in_features=64, out_features=64, bias=True)
  (liner_3): Linear(in_features=64, out_features=1, bias=True)
) 

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    weight_decay: 0
)


In [16]:
loss_fn = nn.BCELoss()
batch = 1024
no_of_batches = len(X)//batch
epochs = 30
from torch.utils.data import TensorDataset, DataLoader


In [17]:
# Ver 4.1 - random_split Mode， charlie
# from torch.utils.data import random_split    
# def accuracy(y_pred, y_true):
#     y_pred = (y_pred > 0.5).type(torch.int32)
#     acc = (y_pred == y_true).float().mean()
#     return acc

# HR_dataset = TensorDataset(X, Y)
# test_ds, test_ds = random_split(HR_dataset, [0.8, 0.2]) 
# HR_dataloader = DataLoader(test_ds, batch_size=batch, shuffle=True)  # shuffle乱序取数
# epochMode = 41
# if epochMode == 41:  
#     for epoch in range( epochs ):
#         # for x,y in HR_dataloader:
#         for x,y in HR_dataloader:
#             y_pred = model(x)
#             loss = loss_fn(y_pred, y)
#             optim.zero_grad()
#             loss.backward()
#             optim.step()
#         with torch.no_grad():
#             print('epoch: ', epoch, ' loss: ', loss_fn(model(X), Y).data.item() )

In [18]:
# Ver 4.2 - sklearn mode
def accuracy(y_pred, y_true):
    y_pred = (y_pred > 0.5).type(torch.int32)
    acc = (y_pred == y_true).float().mean()
    return acc

from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

train_x = torch.from_numpy(train_x.values).type(torch.float32)
train_y = torch.from_numpy(train_y).type(torch.float32)
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=batch, shuffle=True)  # 需要shuffle，乱序执行

test_x = torch.from_numpy(test_x.values).type(torch.float32)
test_y = torch.from_numpy(test_y).type(torch.float32)
test_ds = TensorDataset(test_x, test_y)
test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)  # 不需要shuffle，减少运算量

计算正确率
- Sigmoid()是0和1之间的值， 大于0.5为1，否则是0
- out1 = (y-pred >0.5).type(torch.int32)  : 转换成0或1
- (out1 == labels).float().mean()

In [19]:
# x

In [20]:
%%time
epochs = 100
for epoch in range( epochs ):
    for x,y in train_dl:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
    with torch.no_grad():
        epoch_accuracy = accuracy(model(train_x), train_y)
        epoch_loss = loss_fn(model(train_x), train_y).data.item()
        epoch_test_accuracy = accuracy(model(test_x), test_y)
        epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
        print('epoch: ', 
            epoch, ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
            epoch, ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )
model

epoch:  0  train_loss:  0.5793  Acc： 0.7568 0  test_loss:  0.5662  Acc： 0.773
epoch:  1  train_loss:  0.5671  Acc： 0.7568 1  test_loss:  0.5467  Acc： 0.773
epoch:  2  train_loss:  0.5684  Acc： 0.7568 2  test_loss:  0.5462  Acc： 0.773
epoch:  3  train_loss:  0.5677  Acc： 0.7568 3  test_loss:  0.5498  Acc： 0.773
epoch:  4  train_loss:  0.5667  Acc： 0.7568 4  test_loss:  0.5463  Acc： 0.773
epoch:  5  train_loss:  0.5665  Acc： 0.7568 5  test_loss:  0.5464  Acc： 0.773
epoch:  6  train_loss:  0.5664  Acc： 0.7568 6  test_loss:  0.5475  Acc： 0.773
epoch:  7  train_loss:  0.5661  Acc： 0.7568 7  test_loss:  0.5466  Acc： 0.773
epoch:  8  train_loss:  0.566  Acc： 0.7568 8  test_loss:  0.5461  Acc： 0.773
epoch:  9  train_loss:  0.5658  Acc： 0.7568 9  test_loss:  0.5461  Acc： 0.773
epoch:  10  train_loss:  0.5656  Acc： 0.7568 10  test_loss:  0.546  Acc： 0.773
epoch:  11  train_loss:  0.5655  Acc： 0.7568 11  test_loss:  0.5454  Acc： 0.773
epoch:  12  train_loss:  0.5653  Acc： 0.7568 12  test_loss:  0

Model(
  (liner_1): Linear(in_features=20, out_features=64, bias=True)
  (liner_2): Linear(in_features=64, out_features=64, bias=True)
  (liner_3): Linear(in_features=64, out_features=1, bias=True)
)

In [21]:
# 最后一次计算结果：

epoch_accuracy = accuracy(model(train_x), train_y)
epoch_loss = loss_fn(model(train_x), train_y).data.item()
epoch_test_accuracy = accuracy(model(test_x), test_y)
epoch_test_loss = loss_fn(model(test_x), test_y).data.item()
print('epoch: ', 
    ' train_loss: ', round(epoch_loss,4), ' Acc：', round(epoch_accuracy.item(),4),
    ' test_loss: ', round(epoch_test_loss,4), ' Acc：', round(epoch_test_accuracy.item(),4) )

epoch:   train_loss:  0.5389  Acc： 0.7568  test_loss:  0.5216  Acc： 0.773


In [22]:
# print( loss_fn(model(train_x), train_y).data.item() )
# accuracyRate = ((model(train_x).data.numpy()).astype('int') == train_y.numpy().astype('int')).mean()
# print("训练数据 - 准确率%：", accuracyRate*100) 

# print( loss_fn(model(test_x), test_y).data.item() )
# accuracyRate = ((model(test_x).data.numpy()).astype('int') == test_y.numpy().astype('int')).mean()
# print("测试数据 - 准确率%：", accuracyRate*100) 

- 数据准确性验证
- 过拟合/欠拟合
    - 过拟合：对于已知数据过度拟合，对未知数据预测性差
    - 欠拟合：对于已知数据拟合不够，对未知数据预测性差

In [23]:
# !conda list
# from sklearn.model_selection import train_test_split
# train_x, test_x, train_y, test_y = train_test_split(X_data, Y_data)
# train_x = torch.from_numpy(train_x).type(torch.float32)
# test_x = torch.from_numpy(test_x).type(torch.float32)
# train_y = torch.from_numpy(train_y).type(torch.float32)
# test_y = torch.from_numpy(test_y).type(torch.float32)

In [24]:
# (y_pred == y).float32